# Implementation of N-Gram

## 2-GRAM

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 1
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_1.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_1.pkl", "wb") as f:
    pickle.dump(result, f)

                                                      url  type
301473                    .vbprojects.org/desk/index.html     0
204527                          58.255.19.95:41564/mozi.m     0
726421                         125.45.57.193:60377/Mozi.m     0
443502                       103.217.133.244:37592/Mozi.m     0
524421  panjiva.com/sitemap/supplier_country_detail?co...     1
...                                                   ...   ...
56211   prepechal.net/index.php?option=com_weblinks&vi...     0
370464  absoluteastronomy.com/topics/St._Andrew's_Pres...     1
688432                         115.50.227.57:43679/Mozi.m     0
752565                       seomysite.in/mmn/mduispeiqit     0
580522  hardballtimes.com/main/article/the-virtual-195...     1

[312533 rows x 2 columns]


2023-02-20 14:19:32.339640: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 14:19:32.453081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-20 14:19:32.453105: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-20 14:19:32.475013: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-20 14:19:33.012500: W tensorflow/stream_executor/platform/de

max_tokens 51209
['', '[UNK]', 'c o m', 'h t m', 'i n d', 'n d e', 't m l', 'e n t', 'p h p', 'd e x', 'i o n', 's c o', 't i o', 'e c o', 'e x p', 'n c o', 'x p h', 'a r t', 'o m p', 'i n g', 'o n t', 'o r g', 'i t e', 'o n c', 'c o n', 'l o g', 'v i e', 'a g e', 'n e t', 'n t e', 'o m c', 'e r s', 'a t e', 'a t i', 't o r', 't e r', 'i e w', 'm o z', 'o z i', 'l i n', 'e s t', 't i c', 'a n d', 't h e', 't e m', 'r t i', 'e d i', 's t a', 't e n', 'm c o', 'z i m', 'l c o', 'p t i', 'i n e', '2 0 1', 'o m a', 't c o', 'e m i', 'o p t', 'a l l', 'g i n', 'c a t', 'm i n', 'o n e', 'f o r', 'm i d', 'i s t', 'o m i', 'r e s', 'p r o', 'i k i', 'c l e', 'o m e', 'w i k', 's t o', 'p o p', 'e s s', 'a c o', 'm a r', 'i c l', 'h p o', 'n e w', 'e s c', 'p a g', 'a m p', 's t e', 'e n e', 'a m e', 'o g i', 'y c o', 'd i a', 't i n', 'i d 1', 'o r t', 'm e n', 'r e a', 'w a r', 'o m s', 'e r i', 'o c o', 'i n s', 'r c o', 'w e b', 'o u t', 'l a n', 't r a', 'r i n', 'o m m', 'i n t', 'o m b

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 64)           3277376   
                                                                 
 lstm (LSTM)                 (None, 250, 64)           33024     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,347,649
Trainable params: 3,347,649
Non-

/home/munna0912/anaconda3/envs/amit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: ram://0252c149-fa11-4a19-bf82-17578f92cf43/assets


INFO:tensorflow:Assets written to: ram://0252c149-fa11-4a19-bf82-17578f92cf43/assets


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 2
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_2.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_2.pkl", "wb") as f:
    pickle.dump(result, f)

                                                      url  type
664268                            182.121.252.168:37545/i     0
289515    bulaq.az/images/AolServiceUpgrade/aolupdate.htm     1
470891                             182.114.33.104:41232/i     0
14707                          .kegel.com/corba/lula.html     0
728827  .sv-rotamint.de/index.php/termine/day.listeven...     0
...                                                   ...   ...
225746            gsicunnao.org.in/ian/index.php?QBOT.zip     0
477935  '9d345009-a-62cb3a1a-s-sites.googlegroups.com/...     1
476083  guardian.co.uk/business/2011/jan/19/royal-warr...     1
135562                                antolin-davies.com/     1
447121                         last.fm/music/Krizz+Kaliko     1

[312533 rows x 2 columns]
max_tokens 50024
['', '[UNK]', 'c o m', 'h t m', 'i n d', 'n d e', 't m l', 'p h p', 'e n t', 'd e x', 'i o n', 's c o', 't i o', 'e x p', 'e c o', 'n c o', 'x p h', 'a r t', 'i n g', 'o m p', 'o n t', 'i 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 64)           3201536   
                                                                 
 lstm_2 (LSTM)               (None, 250, 64)           33024     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,271,809
Trainable params: 3,271,809
No

/home/munna0912/anaconda3/envs/amit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: ram://a5ac113c-8cb8-4934-a58a-a27cc8949572/assets


INFO:tensorflow:Assets written to: ram://a5ac113c-8cb8-4934-a58a-a27cc8949572/assets


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 3
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_3.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_3.pkl", "wb") as f:
    pickle.dump(result, f)

                                                      url  type
15393               mcgill.ca/sis/people/faculty/mcnally/     1
44996                        219.154.187.228:45985/Mozi.m     0
320038                        125.46.197.105:34577/bin.sh     0
146392  en.wikipedia.org/wiki/2009_San_Diego_Chargers_...     1
779642                                 5.182.206.101/mips     0
...                                                   ...   ...
383442                             42.230.141.242:37269/i     0
466945            tulumba.com/storeItem.asp?ic=VIFR001116     1
80279                          english.zoo.taipei.gov.tw/     1
113943  directmag.com/mag/marketing_print_quebecor_world/     1
472993  buzzfil.net/article/5635/lol/tetris-la-nouvell...     1

[312533 rows x 2 columns]
max_tokens 49927
['', '[UNK]', 'c o m', 'h t m', 'i n d', 'n d e', 't m l', 'e n t', 'p h p', 'd e x', 'i o n', 's c o', 't i o', 'e c o', 'e x p', 'n c o', 'x p h', 'a r t', 'i n g', 'o m p', 'o n t', 'i 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 250, 64)           3195328   
                                                                 
 lstm_4 (LSTM)               (None, 250, 64)           33024     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,265,601
Trainable params: 3,265,601
No

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 4
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_4.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_4.pkl", "wb") as f:
    pickle.dump(result, f)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 5
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_5.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_5.pkl", "wb") as f:
    pickle.dump(result, f)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 6
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_6.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_6.pkl", "wb") as f:
    pickle.dump(result, f)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 10
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_10.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_10.pkl", "wb") as f:
    pickle.dump(result, f)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 250
Batch_Size = 32
epochs = 15
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe = url_dataframe.sample(frac=0.4)
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" , max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_250_15.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_250_15.pkl", "wb") as f:
    pickle.dump(result, f)